<a href="https://colab.research.google.com/github/issei/DaedalusForge/blob/main/automations/lan%C3%A7amento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automação de Copy para Lançamentos com Gemini e RAG

Este notebook implementa uma rede de 4 agentes de IA para gerar ativos de copy (textos de marketing) para um lançamento de infoproduto. A arquitetura utiliza:

- **LLM**: Google Gemini 1.5 Pro
- **Framework**: LangChain
- **RAG (Retrieval-Augmented Generation)**: DumplingAI como Knowledge Base vetorial.

### Fluxo de Execução
1.  **Configuração**: Instalação de pacotes e definição de chaves de API (Google e Dumpling).
2.  **Ingestão de Dados**: Um briefing de lançamento em formato JSON é carregado.
3.  **Indexação (RAG)**: O briefing é convertido para texto e enviado para uma Knowledge Base (KB) no DumplingAI.
4.  **Retriever Customizado**: Um retriever do LangChain é configurado para consultar a KB do Dumpling via API.
5.  **Rede de Agentes**: 4 agentes especializados são definidos, cada um com uma tarefa específica:
    - `Dores & Promessas`: Identifica os principais problemas e as transformações oferecidas.
    - `Objeções & Quebras`: Antecipa as barreiras de compra e cria argumentos para superá-las.
    - `Headlines & Ângulos`: Gera títulos e abordagens criativas para anúncios e conteúdos.
    - `Adaptação por Canais`: Cria versões do copy adaptadas para diferentes plataformas (Email, Instagram, Anúncios, etc.).
6.  **Orquestração**: Os 4 agentes são executados em paralelo para otimizar o tempo.
7.  **Saída**: Os resultados são agregados, salvos em arquivos `JSON` e `Markdown`, e podem ser opcionalmente reindexados na KB para referência futura.

In [ ]:
!pip -q install -U langchain langchain-google-genai google-generativeai httpx pydantic python-dotenv
print("✅ Dependências instaladas")

✅ Dependências instaladas


In [10]:
import os, json, httpx, re
from typing import List, Optional, Dict, Any
from datetime import datetime
from google.colab import userdata

# LangChain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

# Gemini via LangChain + SDK nativo
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai  # SDK oficial Google

# ======== PARAMS (edite aqui) ========
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
DUMPLING_API_KEY = userdata.get('DUMPLING_API_KEY')
DUMPLING_KB_ID = userdata.get('DUMPLING_KB_ID')
GEMINI_MODEL = "gemini-2.5-flash"  # @param {type:"string"}
TEMPERATURE = 0.6  # @param {type:"number"}
MAX_TOKENS = 2048

assert GOOGLE_API_KEY, "Defina GOOGLE_API_KEY"
assert DUMPLING_API_KEY, "Defina DUMPLING_API_KEY"
assert DUMPLING_KB_ID, "Defina DUMPLING_KB_ID"

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

# ======== Dumpling config ========
DUMPLING_BASE = "https://app.dumplingai.com/api/v1"
DUMPLING_HEADERS = {
    "Authorization": f"Bearer {DUMPLING_API_KEY}",
    "Content-Type": "application/json"
}

print("✅ Chaves configuradas")

✅ Chaves configuradas


In [3]:
# 🧩 Funções auxiliares – DumplingAI (httpx)
class DumplingError(RuntimeError):
    pass

def dumpling_add_to_kb(knowledge_base_id: str, name: str, content: str) -> Dict[str, Any]:
    """
    POST /knowledge-bases/add
    Body: { knowledgeBaseId, name, content }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/add-to-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/add"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "name": name,
        "content": content,
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Add KB falhou: {r.status_code} – {r.text}")
    return r.json()

def dumpling_search_kb(knowledge_base_id: str, query: str, result_count: int = 8) -> List[Dict[str, Any]]:
    """
    POST /knowledge-bases/query
    Body: { knowledgeBaseId, query, resultCount }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/search-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/query"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "query": query,
        "resultCount": max(1, min(int(result_count), 25))
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Query KB falhou: {r.status_code} – {r.text}")
    return r.json()

print("✅ Dumpling helpers prontos")

✅ Dumpling helpers prontos


In [4]:
# 🔎 Retriever customizado
class DumplingRetriever(BaseRetriever):
    knowledge_base_id: str
    top_k: int = 8

    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: Optional[CallbackManagerForRetrieverRun] = None,
    ) -> List[Document]:
        hits = dumpling_search_kb(self.knowledge_base_id, query, self.top_k)
        docs = []
        for h in hits:
            docs.append(Document(
                page_content=h.get("content") or "",
                metadata={
                    "id": h.get("id"),
                    "resource_name": h.get("resource_name"),
                    "similarity": h.get("similarity"),
                }
            ))
        return docs

retriever = DumplingRetriever(knowledge_base_id=DUMPLING_KB_ID, top_k=8)
print("✅ Retriever pronto")

✅ Retriever pronto


In [5]:
BRIEFING_JSON = r'''
{
  "briefing_lancamento": {
    "infoproduto": {
      "nome": "Mentoria de Desenvolvimento Inteligente",
      "produtor": "Mauricio Issei",
      "preco": 99997.00,
      "formato": "Mentoria Individual",
      "descricao": "Mentoria individual para desenvolver arquiteturas de soluções complexas"
    },
    "publico_alvo": {
      "demografia": "Empreendedores digitais iniciantes, profissionais de tecnologia",
      "problema_principal": "Dificuldade em resolver problemas, falta de método para desenvolvimento, baixo faturamento.",
      "transformacao_principal": "Criar soluções de alta qualidade e escalável do zero, alcançando faturamento de 6 ou 7 dígitos.",
      "objecoes_comuns": [
        "Não tenho conhecimento suficiente",
        "O preço é muito alto",
        "Não tenho tempo para aplicar o método"
      ]
    },
    "posicionamento": {
      "diferencial_competitivo": "Único método que combina estratégia de arquitetura de soluções com o poder do desenvolvimento com ferramentas de IA.",
      "tom_de_voz": "Autoridade, inspirador, prático",
      "gatilhos_mentais": [
        "Autoridade",
        "Prova Social",
        "Escassez",
        "Reciprocidade"
      ]
    },
    "estrategia_lancamento": {
      "tipo_lancamento": "Semente",
      "meta_campanha": "Vender 500 unidades e faturar R$ 500.000",
      "datas_chave": {
        "inicio_campanha": "2025-09-15",
        "abertura_carrinho": "2025-09-22",
        "fechamento_carrinho": "2025-09-29"
      },
      "canais": [
        "Email Marketing",
        "Meta Ads",
        "Instagram Stories",
        "YouTube"
      ]
    }
  }
}
'''

briefing = json.loads(BRIEFING_JSON)
print("✅ Briefing carregado")

✅ Briefing carregado


In [6]:
# (Opcional) Upload de arquivo JSON via Colab
try:
    from google.colab import files  # type: ignore
    print("Selecione um arquivo .json (ou pule este passo).")
    up = files.upload()
    if up:
        fname = list(up.keys())[0]
        BRIEFING_JSON = open(fname, "r", encoding="utf-8").read()
        briefing = json.loads(BRIEFING_JSON)
        print("✅ JSON carregado de arquivo:", fname)
except Exception:
    print("Upload não disponível fora do Colab (ok).")

Selecione um arquivo .json (ou pule este passo).


In [7]:
def canonicalize_briefing_to_text(briefing: Dict[str, Any]) -> str:
    b    = briefing.get("briefing_lancamento", {})
    inf  = b.get("infoproduto", {})
    pub  = b.get("publico_alvo", {})
    pos  = b.get("posicionamento", {})
    est  = b.get("estrategia_lancamento", {})
    datas= est.get("datas_chave", {})

    linhas = []
    linhas.append("# Briefing de Lançamento — Canonicalizado")
    linhas.append("## Produto")
    linhas.append(f"Nome: {inf.get('nome','')} | Produtor: {inf.get('produtor','')} | Preço: {inf.get('preco','')} | Formato: {inf.get('formato','')}")
    linhas.append(f"Descrição: {inf.get('descricao','')}")
    linhas.append("\n## Público-alvo & Persona")
    linhas.append(f"Demografia/Psicografia: {pub.get('demografia','')}")
    linhas.append(f"Dor principal: {pub.get('problema_principal','')}")
    linhas.append(f"Transformação: {pub.get('transformacao_principal','')}")
    if pub.get("objecoes_comuns"):
        linhas.append("Objeções comuns:")
        for o in pub["objecoes_comuns"]:
            linhas.append(f"- {o}")
    linhas.append("\n## Posicionamento & Diferencial")
    linhas.append(f"USP: {pos.get('diferencial_competitivo','')}")
    linhas.append(f"Tom de voz: {pos.get('tom_de_voz','')}")
    if pos.get("gatilhos_mentais"):
        linhas.append("Gatilhos prioritários: " + ", ".join(pos["gatilhos_mentais"]))
    linhas.append("\n## Estratégia de Lançamento")
    linhas.append(f"Tipo: {est.get('tipo_lancamento','')} | Meta: {est.get('meta_campanha','')}")
    linhas.append(f"Período/Datas: início={datas.get('inicio_campanha','')} | abertura={datas.get('abertura_carrinho','')} | fechamento={datas.get('fechamento_carrinho','')}")
    if est.get("canais"):
        linhas.append("Canais: " + ", ".join(est["canais"]))
    return "\n".join(linhas)

canonical_text = canonicalize_briefing_to_text(briefing)
kb_name = f"Briefing — {datetime.utcnow().isoformat()}"

resp_add = dumpling_add_to_kb(DUMPLING_KB_ID, kb_name, canonical_text)
print("✅ Briefing indexado na KB com nome:", resp_add.get("name"))

/tmp/ipython-input-3227626404.py:35: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  kb_name = f"Briefing — {datetime.utcnow().isoformat()}"


✅ Briefing indexado na KB com nome: Briefing — 2025-09-10T03:27:51.115700


In [12]:
llm = ChatGoogleGenerativeAI(
    model=GEMINI_MODEL,
    temperature=TEMPERATURE,
    max_output_tokens=MAX_TOKENS
)

SYSTEM_BASE = (
    "Você é parte de uma REDE DE AGENTES especialista em copy para lançamentos de infoprodutos. "
    "Use o TOM DE VOZ do briefing. Foque em clareza, estratégia e ativos prontos para usar. "
    "Respeite a persona (dores, desejos) e o posicionamento (USP)."
)

def join_docs(docs: List[Document]) -> str:
    blocos = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("resource_name") or d.metadata.get("id") or f"doc{i}"
        blocos.append(f"[Fonte: {src}]\n{d.page_content}")
    return "\n\n".join(blocos)

def build_rag_context(brief: Dict[str, Any]) -> str:
    b   = brief.get("briefing_lancamento", {})
    pub = b.get("publico_alvo", {})
    pos = b.get("posicionamento", {})
    est = b.get("estrategia_lancamento", {})

    queries = []
    if pub.get("problema_principal"):
        queries.append(pub["problema_principal"])
    if pub.get("transformacao_principal"):
        queries.append(pub["transformacao_principal"])
    if pos.get("diferencial_competitivo"):
        queries.append(pos["diferencial_competitivo"])
    if est.get("tipo_lancamento"):
        queries.append(f"táticas de lançamento {est['tipo_lancamento']}")

    if not queries:
        queries = ["briefing do produto"]

    docs_all: List[Document] = []
    for q in queries:
        docs_all.extend(retriever.get_relevant_documents(q))

    # dedup simples
    seen, uniq = set(), []
    for d in docs_all:
        key = (d.page_content[:120], d.metadata.get("id"))
        if key not in seen:
            seen.add(key)
            uniq.append(d)

    return join_docs(uniq[:12])

contexto = build_rag_context(briefing)
briefing_str = json.dumps(briefing, ensure_ascii=False, indent=2)

# ===== Agente 1: Dores & Promessas =====
prompt_dores = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Gere DORES priorizadas e PROMESSAS (transformações)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: dores[], promessas[].")
])
chain_dores = prompt_dores | llm

# ===== Agente 2: Objeções & Quebras =====
prompt_obj = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Liste as OBJEÇÕES mais prováveis e QUEBRAS (respostas estratégicas)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: objecoes[], quebras[].")
])
chain_obj = prompt_obj | llm

# ===== Agente 3: Headlines & Ângulos =====
prompt_head = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Proponha HEADLINES e ÂNGULOS criativos prontos para testes."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: headlines[], angulos[].")
])
chain_head = prompt_head | llm

# ===== Agente 4: Adaptação por Canais =====
prompt_canais = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para Canais: Email, Stories IG, Meta Ads, YouTube/VSL."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: email.sequence[], stories.scripts[], ads.variacoes[], vsl.outline[].")
])
chain_canais = prompt_canais | llm

print("✅ Prompts e cadeias prontos")

✅ Prompts e cadeias prontos


In [13]:
parallel = RunnableParallel(
    dores=chain_dores,
    obj=chain_obj,
    head=chain_head,
    canais=chain_canais,
)

outputs = parallel.invoke({
    "briefing": briefing_str,
    "contexto": contexto
})
print("✅ Agentes concluídos")

✅ Agentes concluídos


In [14]:
def safe_json(resp) -> Any:
    text = resp.content if hasattr(resp, "content") else str(resp)
    try:
        return json.loads(text)
    except Exception:
        m = re.search(r"\{[\s\S]*\}\s*$", text)
        if m:
            try:
                return json.loads(m.group(0))
            except Exception:
                pass
        return {"raw": text}

assets = {
    "dores_promessas":  safe_json(outputs["dores"]),
    "objeções_quebras": safe_json(outputs["obj"]),
    "headlines_angulos":safe_json(outputs["head"]),
    "por_canais":       safe_json(outputs["canais"]),
}

import os
os.makedirs("outputs", exist_ok=True)
with open("outputs/assets.json", "w", encoding="utf-8") as f:
    json.dump(assets, f, ensure_ascii=False, indent=2)

with open("outputs/assets.md", "w", encoding="utf-8") as f:
    f.write("# Assets de Lançamento (gerados)\n\n")
    for k,v in assets.items():
        f.write(f"## {k}\n```json\n{json.dumps(v, ensure_ascii=False, indent=2)}\n```\n\n")

print("✅ Salvo em outputs/assets.json e outputs/assets.md")

✅ Salvo em outputs/assets.json e outputs/assets.md


In [ ]:
# @title ⬆️ (Opcional) Enviar os assets gerados para a Knowledge Base
PUSH_ASSETS_TO_KB = False  # @param {type:"boolean"}

if PUSH_ASSETS_TO_KB:
    content_assets = open("outputs/assets.md", "r", encoding="utf-8").read()
    resp_add_assets = dumpling_add_to_kb(
        DUMPLING_KB_ID,
        name=f"Assets Gerados — {datetime.utcnow().isoformat()}",
        content=content_assets
    )
    print("✅ Assets enviados para a KB como:", resp_add_assets.get("name"))
else:
    print("Pulando envio à KB (PUSH_ASSETS_TO_KB=False).")

Pulando envio à KB (PUSH_ASSETS_TO_KB=False).
